# 🎓 AP Shah Institute - Student Access Control System

This notebook demonstrates a comprehensive access control system with three main features:

1. **Strap Detection**: Detects purple AP Shah Institute straps in images
2. **ID Card Recognition**: Extracts student information from ID cards
3. **License Plate Detection**: Identifies vehicle license plates from images and videos

Let's install the required packages first:

In [17]:
# Install required packages
!pip install opencv-python opencv-contrib-python matplotlib pandas numpy ultralytics tqdm

Defaulting to user installation because normal site-packages is not writeable


In [18]:

import os
import json
import cv2
import numpy as np
import pandas as pd
import sqlite3
from datetime import datetime
import matplotlib.pyplot as plt
import re
import easyocr
from IPython.display import display, clear_output
import ipywidgets as widgets
# Cell 2 - Add this after imports
import pytesseract
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
%matplotlib inline
plt.rcParams['figure.figsize'] = (15, 6)

print("Libraries imported successfully")

Libraries imported successfully


In [19]:

BASE_DIR = r"C:\Users\admin\Desktop\Hacknova\License-Plate-Extraction-Save-Data-to-SQL-Database"
ID_CARD_DIR = os.path.join(BASE_DIR, 'id_card_dataset')
STUDENT_DATA_DIR = os.path.join(BASE_DIR, 'student_data')
DB_PATH = os.path.join(BASE_DIR, 'studentDatabase.db')

os.makedirs(STUDENT_DATA_DIR, exist_ok=True)

print(f"Base Directory: {BASE_DIR}")
print(f"ID Cards: {ID_CARD_DIR}")
print(f"Output: {STUDENT_DATA_DIR}")

Base Directory: C:\Users\admin\Desktop\Hacknova\License-Plate-Extraction-Save-Data-to-SQL-Database
ID Cards: C:\Users\admin\Desktop\Hacknova\License-Plate-Extraction-Save-Data-to-SQL-Database\id_card_dataset
Output: C:\Users\admin\Desktop\Hacknova\License-Plate-Extraction-Save-Data-to-SQL-Database\student_data


In [20]:
# Cell 4: Database Class (Fixed)
class StudentDatabase:
    def __init__(self):
        self.db_path = DB_PATH
        self._create_tables()
        
    def _create_tables(self):
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        cursor.execute('''CREATE TABLE IF NOT EXISTS Students (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            student_id TEXT UNIQUE,
            name TEXT,
            department TEXT,
            detection_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            image_path TEXT,
            access_image_path TEXT,
            json_path TEXT
        )''')
        cursor.execute('''CREATE TABLE IF NOT EXISTS AccessLog (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            student_id TEXT,
            access_time TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
            access_granted BOOLEAN,
            FOREIGN KEY (student_id) REFERENCES Students(student_id)
        )''')
        conn.commit()
        conn.close()
        
    def add_student(self, student_id, name, department, image_path, access_image_path, json_path):
        conn = sqlite3.connect(self.db_path)
        cursor = conn.cursor()
        
        # Check if student exists
        cursor.execute("SELECT id FROM Students WHERE student_id = ?", (student_id,))
        if cursor.fetchone():
            cursor.execute('''UPDATE Students SET name=?, department=?, detection_time=CURRENT_TIMESTAMP, 
                image_path=?, access_image_path=?, json_path=? WHERE student_id=?''',
                (name, department, image_path, access_image_path, json_path, student_id))
        else:
            cursor.execute('''INSERT INTO Students (student_id, name, department, image_path, access_image_path, json_path)
                VALUES (?, ?, ?, ?, ?, ?)''', (student_id, name, department, image_path, access_image_path, json_path))
        
        # Log access in same connection
        cursor.execute('INSERT INTO AccessLog (student_id, access_granted) VALUES (?, ?)', (student_id, True))
        
        conn.commit()
        conn.close()

db = StudentDatabase()
print("Database initialized")

Database initialized


In [21]:
# Cell 4.5: Install Tesseract and pytesseract
!pip install pytesseract
# Download Tesseract: https://github.com/UB-Mannheim/tesseract/wiki
# After installing, uncomment and set path:
# import pytesseract
# pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

Defaulting to user installation because normal site-packages is not writeable


In [22]:
# Cell 5: Student ID Card Processor (Simple EasyOCR)

print("Initializing OCR...")
if 'ocr_reader' not in globals():
    import easyocr
    ocr_reader = easyocr.Reader(['en'], gpu=False, verbose=False)
    print("OCR ready!")

class StudentIDProcessor:
    def __init__(self):
        self.db = db
        self.reader = ocr_reader
    
    def extract_info(self, image):
        # Simple preprocessing
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        # Single OCR pass - faster
        results = self.reader.readtext(gray, detail=0)  # detail=0 returns only text, faster
        
        # Extract ID
        student_id = None
        for text in results:
            match = re.search(r'(\d{8,10})', text)
            if match:
                student_id = match.group(1)
                break
        
        # Extract name
        name = None
        skip = {'INSTITUTE', 'TECHNOLOGY', 'ENGINEERING', 'COLLEGE', 'ACADEMIC', 
                'YEAR', 'INFORMATION', 'COMPUTER', 'PRINCIPAL', 'THANE'}
        
        for text in results:
            words = text.upper().split()
            if len(words) == 2:
                if not any(s in words for s in skip):
                    if all(w.isalpha() for w in words):
                        name = text.title()
                        break
        
        # Extract department
        dept_map = {
            'COMPUTER ENGINEERING': 'Computer Engineering',
            'COMPUTER SCIENCE': 'Computer Science',
            'INFORMATION TECHNOLOGY': 'Information Technology',
        }
        
        department = None
        full_text = ' '.join(results).upper()
        for key, val in dept_map.items():
            if key in full_text:
                department = val
                break
        
        return student_id, name, department
    
    def process(self, image_path):
        print(f"Processing: {os.path.basename(image_path)}")
        image = cv2.imread(image_path)
        student_id, name, department = self.extract_info(image)
        
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        file_id = student_id if student_id else f"UNKNOWN_{timestamp}"
        
        # Annotated image
        annotated = image.copy()
        cv2.putText(annotated, f"ID: {student_id or 'N/A'}", (10, 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(annotated, f"Name: {name or 'N/A'}", (10, 60), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(annotated, f"Dept: {department or 'N/A'}", (10, 90), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        
        # Save files
        access_file = f"access_{file_id}_{timestamp}.jpg"
        access_path = os.path.join(STUDENT_DATA_DIR, access_file)
        cv2.imwrite(access_path, annotated)
        
        id_file = f"{file_id}_{timestamp}.jpg"
        id_path = os.path.join(STUDENT_DATA_DIR, id_file)
        cv2.imwrite(id_path, image)
        
        # Save JSON
        json_data = {
            "student_id": student_id,
            "name": name,
            "department": department,
            "detection_time": timestamp,
            "access_image": access_path,
            "id_card_image": id_path
        }
        json_file = f"{file_id}_{timestamp}.json"
        json_path = os.path.join(STUDENT_DATA_DIR, json_file)
        with open(json_path, 'w') as f:
            json.dump(json_data, f, indent=4)
        
        # Save to database
        if student_id:
            self.db.add_student(student_id, name or "N/A", department or "N/A", 
                              id_path, access_path, json_path)
        
        # Display
        fig, axes = plt.subplots(1, 2, figsize=(15, 6))
        axes[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        axes[0].set_title("Original")
        axes[0].axis('off')
        axes[1].imshow(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB))
        axes[1].set_title(f"ID: {student_id or 'N/A'}")
        axes[1].axis('off')
        plt.tight_layout()
        plt.show()
        
        print(f"ID: {student_id or 'Not found'}")
        print(f"Name: {name or 'Not found'}")
        print(f"Dept: {department or 'Not found'}")

processor = StudentIDProcessor()
print("Ready!")

Initializing OCR...
OCR ready!
Ready!


In [ ]:
files = [f for f in os.listdir(ID_CARD_DIR) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

dropdown = widgets.Dropdown(options=files, description='ID Card:', layout=widgets.Layout(width='500px'))
button = widgets.Button(description='Process', button_style='success')
output = widgets.Output()

def on_click(b):
    with output:
        clear_output(wait=True)
        file_path = os.path.join(ID_CARD_DIR, dropdown.value)
        processor.process(file_path)

button.on_click(on_click)
display(widgets.VBox([dropdown, button, output]))

In [24]:
# Cell 7: Install DeepFace - Much easier than face_recognition
!pip install deepface tf-keras opencv-python

print("DeepFace installation complete!")

Defaulting to user installation because normal site-packages is not writeable
DeepFace installation complete!


In [25]:
# Cell 7: Import DeepFace (after installation)
import sys
import importlib

# Force reload the module
try:
    import deepface
    importlib.reload(deepface)
    from deepface import DeepFace
    print("✓ DeepFace imported successfully")
except ImportError:
    print("Installing deepface...")
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "deepface", "tf-keras"])
    from deepface import DeepFace
    print("✓ DeepFace installed and imported")

import pickle

# Create directories
FACE_DB_DIR = os.path.join(BASE_DIR, 'face_database')
os.makedirs(FACE_DB_DIR, exist_ok=True)

print("Face database directory created")

✓ DeepFace imported successfully
Face database directory created


In [26]:
# Cell 8: Extract faces from ALL ID cards
class FaceExtractor:
    def __init__(self):
        self.face_db_dir = FACE_DB_DIR
    
    def extract_face_from_id(self, id_card_path, student_id):
        """Extract and save face from ID card"""
        try:
            # Detect face
            face_objs = DeepFace.extract_faces(
                img_path=id_card_path,
                detector_backend='opencv',
                enforce_detection=False
            )
            
            if face_objs:
                # Get largest face
                face = max(face_objs, key=lambda x: x.get('confidence', 0))
                
                # Load original image
                img = cv2.imread(id_card_path)
                h, w = img.shape[:2]
                
                # Extract face region with padding
                area = face['facial_area']
                padding = 30
                x1 = max(0, area['x'] - padding)
                y1 = max(0, area['y'] - padding)
                x2 = min(w, area['x'] + area['w'] + padding)
                y2 = min(h, area['y'] + area['h'] + padding)
                
                face_img = img[y1:y2, x1:x2]
                
                # Save extracted face
                face_path = os.path.join(self.face_db_dir, f"{student_id}.jpg")
                cv2.imwrite(face_path, face_img)
                
                return face_path
        except Exception as e:
            print(f"Error extracting face: {e}")
        
        return None
    
    def process_all_id_cards(self):
        """Extract faces from all ID cards"""
        files = [f for f in os.listdir(ID_CARD_DIR) 
                if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        
        print(f"Extracting faces from {len(files)} ID cards...\n")
        
        extracted = 0
        for i, file in enumerate(files, 1):
            file_path = os.path.join(ID_CARD_DIR, file)
            
            # Get student ID using OCR
            image = cv2.imread(file_path)
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            results = ocr_reader.readtext(gray, detail=0)
            
            student_id = None
            for text in results:
                match = re.search(r'(\d{8,10})', text)
                if match:
                    student_id = match.group(1)
                    break
            
            if not student_id:
                student_id = os.path.splitext(file)[0]  # Use filename
            
            print(f"[{i}/{len(files)}] {file} -> ID: {student_id}")
            
            face_path = self.extract_face_from_id(file_path, student_id)
            
            if face_path:
                print(f"   ✓ Face saved: {os.path.basename(face_path)}")
                extracted += 1
            else:
                print(f"   ✗ No face detected")
            
            print()
        
        print(f"Extraction complete! {extracted}/{len(files)} faces extracted")
        return extracted

extractor = FaceExtractor()
print("Face Extractor ready!")

Face Extractor ready!


In [27]:
# Cell 9: Extract all faces NOW
extractor.process_all_id_cards()

Extracting faces from 64 ID cards...

[1/64] abhishek.jpg -> ID: 22104159
   ✓ Face saved: 22104159.jpg

[2/64] anish.jpg -> ID: anish
   ✓ Face saved: anish.jpg

[3/64] ashmit.jpg -> ID: 24104139
   ✓ Face saved: 24104139.jpg

[4/64] avanish.jpg -> ID: 22102003
   ✓ Face saved: 22102003.jpg

[5/64] harsh.jpg -> ID: 24102107
   ✓ Face saved: 24102107.jpg

[6/64] ishan.jpg -> ID: 24102083
   ✓ Face saved: 24102083.jpg

[7/64] lucky.jpg -> ID: 22102121
   ✓ Face saved: 22102121.jpg

[8/64] manthan.jpg -> ID: 22102124
   ✓ Face saved: 22102124.jpg

[9/64] mohika.jpg -> ID: 22102164
   ✓ Face saved: 22102164.jpg

[10/64] prathmesh.jpg -> ID: 24106107
   ✓ Face saved: 24106107.jpg

[11/64] samay.jpg -> ID: 23202005
   ✓ Face saved: 23202005.jpg

[12/64] strap_image 2 copy.jpg -> ID: 24106107
   ✓ Face saved: 24106107.jpg

[13/64] strap_image 2.jpg -> ID: 24106107
   ✓ Face saved: 24106107.jpg

[14/64] strap_image_purple_apshah copy.jpg -> ID: 22102121
   ✓ Face saved: 22102121.jpg

[15/64] 

64

In [13]:
# Cell 10: Debug Face Recognition with Visual Comparison
import os
import cv2
import json
import sqlite3
from datetime import datetime
import numpy as np
from deepface import DeepFace
import matplotlib.pyplot as plt

BASE_DIR = r"C:\Users\admin\Desktop\Hacknova\License-Plate-Extraction-Save-Data-to-SQL-Database"
FACE_DB_DIR = os.path.join(BASE_DIR, 'face_database')
STUDENT_DATA_DIR = os.path.join(BASE_DIR, 'student_data')
DB_PATH = os.path.join(BASE_DIR, 'studentDatabase.db')

class DebugEntrySystem:
    def __init__(self):
        self.face_db_dir = FACE_DB_DIR
        self.db_path = DB_PATH
        self.student_data_dir = STUDENT_DATA_DIR
        self.known_faces = self._load_faces()
        self.student_info = self._load_student_info()
        
        # Manually add your info if missing
        if '22102035' not in self.student_info:
            self.student_info['22102035'] = {
                "name": "Dipesh Sharma",
                "dept": "Computer Science"
            }
            print("Added Dipesh Sharma to student info")
    
    def _load_faces(self):
        faces = {}
        for file in os.listdir(self.face_db_dir):
            if file.endswith('.jpg'):
                student_id = os.path.splitext(file)[0]
                faces[student_id] = os.path.join(self.face_db_dir, file)
        print(f"Loaded {len(faces)} faces")
        return faces
    
    def _load_student_info(self):
        info = {}
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            cursor.execute("SELECT student_id, name, department FROM Students")
            for row in cursor.fetchall():
                if row[1] and row[1] != "N/A":
                    info[row[0]] = {"name": row[1], "dept": row[2] or "N/A"}
            conn.close()
        except:
            pass
        return info
    
    def verify_and_visualize(self, live_path, db_path, student_id):
        """Verify and show side-by-side comparison"""
        try:
            result = DeepFace.verify(
                img1_path=live_path,
                img2_path=db_path,
                model_name='VGG-Face',  # More lenient
                enforce_detection=False,
                distance_metric='cosine'
            )
            
            # Show comparison
            fig, axes = plt.subplots(1, 2, figsize=(10, 5))
            
            live_img = cv2.cvtColor(cv2.imread(live_path), cv2.COLOR_BGR2RGB)
            db_img = cv2.cvtColor(cv2.imread(db_path), cv2.COLOR_BGR2RGB)
            
            axes[0].imshow(live_img)
            axes[0].set_title('Live Camera', fontsize=14, fontweight='bold')
            axes[0].axis('off')
            
            axes[1].imshow(db_img)
            axes[1].set_title(f'Database: {student_id}', fontsize=14, fontweight='bold')
            axes[1].axis('off')
            
            match = "MATCH ✓" if result['verified'] else "NO MATCH ✗"
            color = 'green' if result['verified'] else 'red'
            fig.suptitle(f'{match} | Distance: {result["distance"]:.4f} | Threshold: {result["threshold"]:.4f}',
                        fontsize=16, fontweight='bold', color=color)
            
            plt.tight_layout()
            plt.show()
            
            return result['verified'], result['distance']
        except Exception as e:
            print(f"Error: {e}")
            return False, 1.0
    
    def find_match(self, live_face_path):
        """Find best match with detailed output"""
        print("\nChecking all faces...")
        
        matches = []
        
        for student_id, face_path in self.known_faces.items():
            try:
                result = DeepFace.verify(
                    img1_path=live_face_path,
                    img2_path=face_path,
                    model_name='VGG-Face',
                    enforce_detection=False,
                    distance_metric='cosine'
                )
                
                distance = result['distance']
                verified = result['verified']
                
                if distance < 0.7:  # Show all reasonable matches
                    matches.append((student_id, distance, verified))
                    print(f"  {student_id}: distance={distance:.4f}, verified={verified}")
            except:
                continue
        
        if matches:
            # Sort by distance (lower is better)
            matches.sort(key=lambda x: x[1])
            best_id, best_dist, best_verified = matches[0]
            
            print(f"\nBest match: {best_id} (distance: {best_dist:.4f})")
            
            # Show visual comparison for best match
            self.verify_and_visualize(live_face_path, self.known_faces[best_id], best_id)
            
            # Accept if distance < 0.6 (more lenient)
            if best_dist < 0.6:
                return best_id, best_dist
        
        return None, 1.0
    
    def log_entry(self, student_id, name, confidence):
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        
        entry_log = {
            "student_id": student_id,
            "name": name,
            "entry_time": timestamp,
            "confidence": f"{(1-confidence)*100:.1f}%"
        }
        
        json_path = os.path.join(self.student_data_dir, f"entry_{student_id}_{timestamp}.json")
        with open(json_path, 'w') as f:
            json.dump(entry_log, f, indent=4)
        
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            cursor.execute('INSERT INTO AccessLog (student_id, access_granted) VALUES (?, ?)',
                          (student_id, True))
            conn.commit()
            conn.close()
        except:
            pass
    
    def start(self):
        print("\n" + "="*60)
        print("DEBUG FACE RECOGNITION - VISUAL MODE")
        print("="*60)
        print("Press SPACE to scan | Q to quit\n")
        
        cap = cv2.VideoCapture(0)
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
        
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            display = frame.copy()
            cv2.putText(display, "Press SPACE to scan face", (10, 30),
                      cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 0), 2)
            cv2.putText(display, "Dipesh - look here!", (10, 60),
                      cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            
            cv2.imshow('Debug System - SPACE=scan, Q=quit', display)
            
            key = cv2.waitKey(1) & 0xFF
            
            if key == ord('q'):
                break
            
            elif key == ord(' '):
                temp_file = "temp_debug.jpg"
                cv2.imwrite(temp_file, frame)
                
                matched_id, distance = self.find_match(temp_file)
                
                if matched_id:
                    info = self.student_info.get(matched_id, {"name": matched_id, "dept": "N/A"})
                    confidence = 1 - distance
                    
                    self.log_entry(matched_id, info["name"], distance)
                    
                    print(f"\n{'='*60}")
                    print(f"✓ ACCESS GRANTED")
                    print(f"Name: {info['name']}")
                    print(f"ID: {matched_id}")
                    print(f"Confidence: {confidence*100:.1f}%")
                    print(f"{'='*60}\n")
                    
                    success = frame.copy()
                    cv2.rectangle(success, (0, 0), (640, 120), (0, 255, 0), -1)
                    cv2.putText(success, "ACCESS GRANTED", (20, 40),
                              cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                    cv2.putText(success, f"{info['name']}", (20, 70),
                              cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
                    cv2.putText(success, f"ID: {matched_id} | {confidence*100:.1f}%", 
                              (20, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                    
                    cv2.imshow('Debug System - SPACE=scan, Q=quit', success)
                    cv2.waitKey(3000)
                else:
                    print("\n✗ ACCESS DENIED: No match found\n")
                    
                    denied = frame.copy()
                    cv2.rectangle(denied, (0, 0), (640, 80), (0, 0, 255), -1)
                    cv2.putText(denied, "ACCESS DENIED", (20, 50),
                              cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 255, 255), 3)
                    cv2.imshow('Debug System - SPACE=scan, Q=quit', denied)
                    cv2.waitKey(2000)
                
                if os.path.exists(temp_file):
                    os.remove(temp_file)
        
        cap.release()
        cv2.destroyAllWindows()
        print("System stopped")

system = DebugEntrySystem()

Loaded 51 faces
Added Dipesh Sharma to student info


In [9]:
system = StableEntrySystem()

Loaded 51 faces


In [10]:
entry_system = StableEntrySystem()

Loaded 51 faces


In [14]:
entry_system.start()


FACE RECOGNITION ENTRY
Press SPACE to scan, Q to quit

Searching database...
Checking 4/51: 22102025

ACCESS: Unknown (22102025) - 87.4%

Searching database...
Checking 51/51: WhatsApp Image 2025-10-03 at 22.20.08_fa9ddabc

DENIED: Face not recognized

Searching database...
Checking 4/51: 22102025

ACCESS: Unknown (22102025) - 81.6%

Searching database...
Checking 51/51: WhatsApp Image 2025-10-03 at 22.20.08_fa9ddabc

DENIED: Face not recognized

Searching database...
Checking 4/51: 22102025

ACCESS: Unknown (22102025) - 88.9%

Searching database...
Checking 4/51: 22102025

ACCESS: Unknown (22102025) - 97.4%

Searching database...
Checking 4/51: 22102025

ACCESS: Unknown (22102025) - 96.1%

Searching database...
Checking 3/51: 22102003

ACCESS: Unknown (22102003) - 68.0%

Searching database...
Checking 51/51: WhatsApp Image 2025-10-03 at 22.20.08_fa9ddabc

DENIED: Face not recognized



KeyboardInterrupt: 